# SPEECH EMOTION RECOGNITION

### IMPORTING LIBRARIES

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import keras
import os
import os.path
import sys

import librosa                                                          #To extract the data from the audio files.
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier                      #For KNN
from sklearn.svm import SVC                                             #For SVM
from sklearn.preprocessing import LabelEncoder                          #For Naives Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from IPython.display import Audio                                       #To play the audio files

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### PREPROCESSING THE DATA

#### Defining the path of the dataset files.

In [17]:
Ravdess = "/Users/sanjanathakal/Project/archive/audio_speech_actors_01-24/"
Crema = "/Users/sanjanathakal/Project/AudioWAV/"
Tess = "/Users/sanjanathakal/Project/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/"
Savee = "/Users/sanjanathakal/Project/ALL"

In [26]:
##For dataset 1 : RAVDESS
import os
import pandas as pd

Ravdess_Dir_list = os.listdir(Ravdess)
#print(Ravdess_Dir_list)
emotionfile = []
pathfile = []
i=0
for i in Ravdess_Dir_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    newpath = os.path.join(Ravdess,i)
    print(newpath)
    for j in newpath:
        part = j.split('.')[0]
        print(part)
       #"""part = part.split('-')
       #third part in each file represents the emotion associated to that file.
        #emotionfile.append(int(part[2]))
        #pathfile.append(Ravdess + i + '/' + j)"""
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(emotionfile, columns=['Emotions'])
print(emotion_df)

# dataframe for path of files.
path_df = pd.DataFrame(pathfile, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

/Users/sanjanathakal/Project/archive/audio_speech_actors_01-24/Actor_16
/
U
s
e
r
s
/
s
a
n
j
a
n
a
t
h
a
k
a
l
/
P
r
o
j
e
c
t
/
a
r
c
h
i
v
e
/
a
u
d
i
o
_
s
p
e
e
c
h
_
a
c
t
o
r
s
_
0
1
-
2
4
/
A
c
t
o
r
_
1
6
/Users/sanjanathakal/Project/archive/audio_speech_actors_01-24/Actor_11
/
U
s
e
r
s
/
s
a
n
j
a
n
a
t
h
a
k
a
l
/
P
r
o
j
e
c
t
/
a
r
c
h
i
v
e
/
a
u
d
i
o
_
s
p
e
e
c
h
_
a
c
t
o
r
s
_
0
1
-
2
4
/
A
c
t
o
r
_
1
1
/Users/sanjanathakal/Project/archive/audio_speech_actors_01-24/Actor_18
/
U
s
e
r
s
/
s
a
n
j
a
n
a
t
h
a
k
a
l
/
P
r
o
j
e
c
t
/
a
r
c
h
i
v
e
/
a
u
d
i
o
_
s
p
e
e
c
h
_
a
c
t
o
r
s
_
0
1
-
2
4
/
A
c
t
o
r
_
1
8
/Users/sanjanathakal/Project/archive/audio_speech_actors_01-24/Actor_20
/
U
s
e
r
s
/
s
a
n
j
a
n
a
t
h
a
k
a
l
/
P
r
o
j
e
c
t
/
a
r
c
h
i
v
e
/
a
u
d
i
o
_
s
p
e
e
c
h
_
a
c
t
o
r
s
_
0
1
-
2
4
/
A
c
t
o
r
_
2
0
/Users/sanjanathakal/Project/archive/audio_speech_actors_01-24/Actor_21
/
U
s
e
r
s
/
s
a
n
j
a
n
a
t
h
a
k
a
l
/
P
r
o
j
e
c
t
/
a
r
c
h
i
v
e


,Emotions,Path
